## Test the Retrieval Latency of Approximate vs Exact Matching 

In [ ]:
import tensorflow as tf
import time

In [ ]:
PROJECT_ID = 'ksalama-cloudml'
BUCKET = 'ksalama-cloudml'
INDEX_DIR = f'gs://{BUCKET}/bqml/scann_index'
BQML_MODEL_DIR = f'gs://{BUCKET}/bqml/item_matching_model'
LOOKUP_MODEL_DIR = f'gs://{BUCKET}/bqml/embedding_lookup_model'

## Exact Matching

In [ ]:
class ExactMatcher(object):
  def __init__(self, model_dir):
    print("Loading exact matchg model...")
    self.model = tf.saved_model.load(model_dir)
    print("Exact matchg model is loaded.")
  
  def match(self, instances):
    outputs = self.model.signatures['serving_default'](tf.constant(instances, tf.dtypes.int64))
    return outputs['predicted_item2_Id'].numpy()

In [ ]:
exact_matcher = ExactMatcher(BQML_MODEL_DIR)

In [ ]:
start_time = time.time()
for i in range(100):
  exact_matcher.match([2114402])
end_time = time.time()
elapsed_time = end_time - start_time

print(f'Elapsed time:{elapsed_time} - average time: {elapsed_time/100}')

## Approximate Matching (ScaNN)

In [ ]:
from index_server.matching import ScaNNMatcher
scann_matcher = ScaNNMatcher(INDEX_DIR)
embedding_lookup = tf.saved_model.load(LOOKUP_MODEL_DIR)

In [ ]:
start_time = time.time()
for i in range(100):
  vector = embedding_lookup(['2114402']).numpy()[0]
  scann_matcher.match(vector, 50)
end_time = time.time()
elapsed_time = end_time - start_time

print(f'Elapsed time:{elapsed_time} - average time: {elapsed_time/100}')

## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**